In [1]:
# import Libraries

import pandas as pd
import os
import folium

In [2]:
boarding_info = pd.read_csv('boarding_info.csv', encoding='utf-8')

In [5]:
days = set(boarding_info['Day'])
stops = set(boarding_info['Stopno'])

In [6]:
avg_ = {
    'line' : [],
    'stopno' : [],
    'on' : [],
    'off' : []
}

In [7]:
for s in stops:
    stop_info = boarding_info[boarding_info['Stopno']==s]
    lines = set(stop_info['Line'])
    for l in lines:
        line_info = stop_info[stop_info['Line']==l]
        on_ = 0
        off_ = 0
        for i, row in line_info.iterrows():
            on_ += int(row['geton'])
            off_ += int(row['off'])
        avg_on = on_/len(line_info)
        avg_off = off_/len(line_info)
        avg_['line'].append(l)
        avg_['stopno'].append(s)
        avg_['on'].append(avg_on)
        avg_['off'].append(avg_off)


In [9]:
boarding_avg = pd.DataFrame(avg_)

In [15]:
boarding_avg.to_csv('boarding_avg.csv')

In [2]:
## plase start at this code after you save boarding_avg.csv
boarding_avg = pd.read_csv('boarding_avg.csv')

In [60]:
stop_loc = pd.read_csv('busstop_loc2.csv')

In [62]:
df_fin = stop_loc.join(boarding_avg.set_index('stopno'), on='stop_id')

In [63]:
df_fin = df_fin.drop(columns=['Unnamed: 0']).dropna(0)

In [110]:
stops = set(df_fin['stop_id'])

{1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1037,
 1041,
 1044,
 1101,
 1102,
 1103,
 1104,
 1105,
 1106,
 1107,
 1108,
 1109,
 1110,
 1111,
 1112,
 1113,
 1114,
 1115,
 1116,
 1117,
 1118,
 1120,
 1123,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1132,
 1133,
 1134,
 1135,
 1136,
 1137,
 1138,
 1139,
 1140,
 1141,
 1142,
 1143,
 1144,
 1146,
 1148,
 1149,
 1150,
 1151,
 1152,
 1153,
 1154,
 1155,
 1156,
 1157,
 1158,
 1159,
 1160,
 1161,
 1162,
 1163,
 1164,
 1165,
 1167,
 1168,
 1169,
 1171,
 1172,
 1173,
 1174,
 1175,
 1176,
 1177,
 1198,
 1199,
 1200,
 1201,
 1202,
 1203,
 1204,
 1205,
 1212,
 1213,
 1214,
 1215,
 1217,
 1218,
 1219,
 1220,
 1221,
 1222,
 1223,
 1224,
 1225,
 1226,
 1227,
 1228,
 1229,
 1231,
 1235,
 1236,
 1238,
 1239,
 1240,
 1243,
 1245,
 1246,
 1247,
 1249,
 1250,
 1251,
 1252,
 1253,
 1254,
 1256,
 1262,
 1267,
 1272,
 1277,
 1278,
 1279,

In [106]:
total_ = {
    'stop_id' : [],
    'stop_name' : [],
    'on' : [],
    'off' : [],
    'x' :[],
    'y' : []
}

In [108]:
for s in stops:
    stop_info = df_fin[df_fin['stop_id']==s]
    on_ = 0
    off_ = 0
    name_ = df_fin[df_fin['stop_id']==s].iloc[0]['stop_name']
    x = df_fin[df_fin['stop_id']==s].iloc[0]['x']
    y = df_fin[df_fin['stop_id']==s].iloc[0]['y']
    for i, row in stop_info.iterrows():
        on_ += row['on']
        off_ += row['off']
    total_['stop_id'].append(s)
    total_['stop_name'].append(name_)
    total_['on'].append(int(on_))
    total_['off'].append(int(off_))
    total_['x'].append(x)
    total_['y'].append(y)

In [111]:
total_df = pd.DataFrame(total_)

In [121]:
max_ = total_df['on'].max()
min_ = total_df['on'].min()

In [154]:
map_on = folium.Map(location=[37.566047, 126.977702], tiles='Stamen Toner', zoom_start=15)

In [155]:
for i, row in total_df.iterrows():
    location = [float(row['y']),float(row['x'])]
    folium.Circle(
        location = location,
        radius = int((row['off']/max_)*200) ,
        color='ffffff',
        fill_opacity=0.7,
        fill_color='#F44336',
        popup = 'Bus stop : ' + str(row['stop_name']) + '<br>' +
        'Get on : ' + str("{:,} person/day".format(int(row['on']))) + '<br>' +
        'Get off : ' + str("{:,} person/day".format(int(row['off']))),
        fill=True
    ).add_to(map_on)

In [156]:
for i, row in total_df.iterrows():
    location = [float(row['y']),float(row['x'])]
    folium.Circle(
        location = location,
        radius = int((row['on']/max_)*200) ,
        color='ffffff',
        fill_opacity=0.7,
        fill_color='#2196F3',
        popup = 'Bus stop : ' + str(row['stop_name']) + '<br>' +
        'Get on : ' + str("{:,} person/day".format(int(row['on']))) + '<br>' +
        'Get off : ' + str("{:,} person/day".format(int(row['off']))),
        fill=True
    ).add_to(map_on)

In [157]:
map_on.save('test.html')